**SOFASCORE** (https://www.sofascore.com)

Sofascore is a website that provides live scores, results, and fixtures for a variety of sports. It also provides statistics and player ratings for football matches.

It has some advanced statistics and event data we can use like shots and expected goals.

The unique thing about Sofascore is that it is a website that is set up terribly for scraping, but we can still do it thanks to APIs they use to load the data.

In [1]:
import json

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# let's test if web driver is working correctly and it's able to open a Chrome window
driver=webdriver.Chrome()

driver.get("https://www.google.com/")

In [10]:
# settings
options = webdriver.ChromeOptions()
options.set_capability(
    "goog:loggingPrefs", {"performance": "ALL", "browser": "ALL"}
)

# make sure you already have Chrome installed
driver = webdriver.Chrome(options=options)
# don't load the page longer than 10 seconds to avoid infinite loop
driver.set_page_load_timeout(10)

try:
    driver.get("https://www.sofascore.com/football/match/bolivia-venezuela/xUbsWUb#id:12716820,tab:statistics")
except:
    pass

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [11]:
# extract requests from logs | this gives us all the apis that are being call
logs_raw = driver.get_log("performance")

In [12]:
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

In [13]:
# dictionaries
logs[0]

{'method': 'Network.policyUpdated', 'params': {}}

In [14]:
# looping through every network response
for x in logs:
    if 'shotmap' in x['params'].get('headers', {}).get(':path', ''):
        print(x['params'].get('headers', {}).get(':path'))
        break

/api/v1/event/12716820/shotmap


In [15]:
# information about the network response
x

{'method': 'Network.requestWillBeSentExtraInfo',
 'params': {'associatedCookies': [],
  'clientSecurityState': {'initiatorIPAddressSpace': 'Public',
   'initiatorIsSecureContext': True,
   'privateNetworkRequestPolicy': 'PreflightWarn'},
  'connectTiming': {'requestTime': 11527.46641},
  'headers': {':authority': 'www.sofascore.com',
   ':method': 'GET',
   ':path': '/api/v1/event/12716820/shotmap',
   ':scheme': 'https',
   'accept': '*/*',
   'accept-encoding': 'gzip, deflate, br, zstd',
   'accept-language': 'en-US,en;q=0.9',
   'cache-control': 'max-age=0',
   'priority': 'u=1, i',
   'referer': 'https://www.sofascore.com/football/match/bolivia-venezuela/xUbsWUb',
   'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
   'sec-ch-ua-mobile': '?0',
   'sec-ch-ua-platform': '"Linux"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-origin',
   'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, l

In [16]:
shotmap = json.loads(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': x["params"]["requestId"]})['body'])['shotmap']
shotmap[0]

{'player': {'name': 'Enzo Monteiro',
  'firstName': 'Enzo Monteiro',
  'slug': 'enzo-monteiro',
  'shortName': 'E. Monteiro',
  'position': 'F',
  'jerseyNumber': '18',
  'userCount': 194,
  'id': 1525129},
 'isHome': True,
 'shotType': 'goal',
 'goalType': 'regular',
 'situation': 'assisted',
 'playerCoordinates': {'x': 3.8, 'y': 49.2, 'z': 0},
 'bodyPart': 'right-foot',
 'goalMouthLocation': 'high-right',
 'goalMouthCoordinates': {'x': 0, 'y': 45.8, 'z': 23.4},
 'id': 3725670,
 'time': 89,
 'timeSeconds': 5304,
 'draw': {'start': {'x': 49.2, 'y': 3.8},
  'end': {'x': 54.2, 'y': 0},
  'goal': {'x': 54.2, 'y': 76.6}},
 'reversedPeriodTime': 2,
 'reversedPeriodTimeSeconds': 96,
 'incidentType': 'shot'}

In [17]:
shotmap

[{'player': {'name': 'Enzo Monteiro',
   'firstName': 'Enzo Monteiro',
   'slug': 'enzo-monteiro',
   'shortName': 'E. Monteiro',
   'position': 'F',
   'jerseyNumber': '18',
   'userCount': 194,
   'id': 1525129},
  'isHome': True,
  'shotType': 'goal',
  'goalType': 'regular',
  'situation': 'assisted',
  'playerCoordinates': {'x': 3.8, 'y': 49.2, 'z': 0},
  'bodyPart': 'right-foot',
  'goalMouthLocation': 'high-right',
  'goalMouthCoordinates': {'x': 0, 'y': 45.8, 'z': 23.4},
  'id': 3725670,
  'time': 89,
  'timeSeconds': 5304,
  'draw': {'start': {'x': 49.2, 'y': 3.8},
   'end': {'x': 54.2, 'y': 0},
   'goal': {'x': 54.2, 'y': 76.6}},
  'reversedPeriodTime': 2,
  'reversedPeriodTimeSeconds': 96,
  'incidentType': 'shot'},
 {'player': {'name': 'Cristian Cásseres Jr.',
   'slug': 'cristian-casseres-jr',
   'shortName': 'C. Cásseres Jr.',
   'position': 'M',
   'jerseyNumber': '23',
   'userCount': 1000,
   'id': 878938,
   'fieldTranslations': {'nameTranslation': {'ar': 'Casseres Jr